In [32]:
from PIL import Image
import numpy as np
import math
import random
from PIL import ImageOps
file = open('model_1.obj', 'r')
v = []
f = []
z_buffer = np.full((1000,1000), np.inf, dtype = np.float32)
img_mat = np.zeros((1000, 1000, 3), dtype = np.uint8)
for str in file:
    splitted_str = str.split()
    if(splitted_str[0] == 'v'):
        v.append((float(splitted_str[1]),float(splitted_str[2]),float(splitted_str[3])))
    if(splitted_str[0] == 'f'):
        f.append((int(splitted_str[1].split('/')[0]),int(splitted_str[2].split('/')[0]),int(splitted_str[3].split('/')[0])))
        
for h in range(len(v)):
    v[h] = rotate(0,-15,0,v[h])

for i in range(len(f)):
    
    x0 = v[f[i][0]-1][0]
    y0 = v[f[i][0]-1][1]
    z0 = v[f[i][0]-1][2]
    x1 = v[f[i][1]-1][0]
    y1 = v[f[i][1]-1][1]
    z1 = v[f[i][1]-1][2]
    x2 = v[f[i][2]-1][0]
    y2 = v[f[i][2]-1][1]
    z2 = v[f[i][2]-1][2]

    n = normale(x0,y0,z0,x1,y1,z1,x2,y2,z2)
    cosA = scalar(n)
    if (cosA < 0):
        triangle_infill(x0,y0,x1,y1,x2,y2, img_mat, cosA, z0, z1, z2, z_buffer)
    # x_loop_finalll(img_mat, x0, y0, x1, y1)
    # x_loop_finalll(img_mat, x1, y1, x2, y2)
    # x_loop_finalll(img_mat, x2, y2, x0, y0)
img = Image.fromarray(img_mat, mode='RGB')
im = ImageOps.flip(img)
im.save("rabbit_rotate_value.png")

In [31]:
def triangle_infill(x0, y0, x1, y1, x2, y2, canvas, cosA, z0, z1, z2, z_buffer):
    
    x0_p = x0 * 1500/z0 + 500
    y0_p = y0 * 1500/z0 + 500
    x1_p = x1 * 1500/z1 + 500
    y1_p = y1 * 1500/z1 + 500
    x2_p = x2 * 1500/z2 + 500
    y2_p = y2 * 1500/z2 + 500
        
    xmin = min(x0_p, x1_p, x2_p)-1
    ymin = min(y0_p, y1_p, y2_p)-1
    xmax = max(x0_p, x1_p, x2_p)+1
    ymax = max(y0_p, y1_p, y2_p)+1
    
    if(xmin < 0):
        xmin = 0
    if(ymin < 0):
        ymin = 0
    for x in range(math.ceil(xmin), math.floor(xmax)):
        for y in range(math.ceil(ymin), math.floor(ymax)):
            lambd0, lambd1, lambd2 = bar_coord(x0_p, y0_p, x1_p, y1_p, x2_p, y2_p, x, y)
            
            if (lambd0 >= 0 and lambd1 >= 0 and lambd2 >= 0): 
                z_display= lambd0*z0 + lambd1*z1 + lambd2*z2
                if(z_buffer[y,x] > z_display):
                    canvas[y,x] = (-255 * cosA, -255 * cosA, -255 * cosA) 
                    z_buffer[y,x] = z_display

    

In [13]:
#Барицентрические координаты 1

def bar_coord(x0, y0, x1, y1, x2, y2, x, y):
    lambd0 = ((x - x2) * (y1 - y2) - (x1 - x2) * (y - y2)) /  ((x0 - x2) * (y1 - y2) - (x1 - x2) * (y0 - y2)) 
    lambd1 = ((x0 - x2) * (y - y2) - (x - x2) * (y0 - y2)) /  ((x0 - x2) * (y1 - y2) - (x1 - x2) * (y0 - y2)) 
    lambd2 = 1.0 - lambd0 - lambd1
    return lambd0, lambd1, lambd2

In [14]:
# вычисление нормали

def normale(x0,y0,z0,x1,y1,z1,x2,y2,z2):
    n = [0] * 3
    n[0] = ((y1-y2) * (z1-z0)) - ((z1-z2) * (y1-y0))
    n[1] = ((x1-x2) * (z1-z0)) - ((z1-z2) * (x1-x0))
    n[2] = ((x1-x2) * (y1-y0)) - ((y1-y2) * (x1-x0))
    return n

In [15]:
# скалярное произведение

def scalar(n):
    cosA = n[2]/ math.sqrt(n[0]**2 + n[1]**2 + n[2]**2)
    return cosA

In [26]:
#поворот модели
def rotate(a, b, c, v):
    
    shift = np.array([0,-0.03,0.2])
    matrixRa = np.array([[1,0,0],[0, math.cos(a),math.sin(a)],[0,-1*math.sin(a), math.cos(a)]])
    matrixRb = np.array([[math.cos(b),0,math.sin(b)],[0,1,0],[-1*math.sin(b),0,math.cos(b)]])
    matrixRc = np.array([[math.cos(c),math.sin(c),0],[-1*math.sin(c),math.cos(c),0],[0,0,1]])
    R = matrixRa @ matrixRb @ matrixRc
    result = np.dot(R, v) + shift
    return result

In [17]:
# проективное преобразования
def transf(v):
    vect = []